# Kaggle : Airbnb New User Bookings

In [ ]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import seaborn as sns

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

# data

# sessions

In [ ]:
#10M data
sessions = pd.read_csv("~/Documents/data/airbnb/sessions.csv")
sessions.head()

In [ ]:
sessions.isnull().sum()

In [ ]:
#359 action type
sessions['action'].value_counts()

In [ ]:
#155 action detail
sessions['action_detail'].value_counts()

In [ ]:
#10 action type
sessions['action_type'].value_counts()

In [ ]:
# sns.distplot(sessions['secs_elapsed']);
# plt.show()

# age_gender_bkts

In [ ]:
#21 age bucket (5 years) per gender by country
#1K unit
age_gender = pd.read_csv("~/Documents/data/airbnb/age_gender_bkts.csv")
age_gender.head()

In [ ]:
age_gender.info()

# countries (destination)

In [ ]:
#AU = austrailia
#CA = canada
#DE = germany
#ES = spain
#FR = france
#GB = england
#IT = italia
#NL = netherland 
#PT = Portugal
#US = USA

countries = pd.read_csv("~/Documents/data/airbnb/countries.csv")
countries

In [ ]:
countries['country_destination'].value_counts()

# submission

In [ ]:
# submission = pd.read_csv("~/Documents/data/airbnb/sample_submission_NDF.csv")
# submission.head()

In [ ]:
# submission.info()

# test

In [ ]:
test = pd.read_csv("~/Documents/data/airbnb/test_users.csv")
test.head()

In [ ]:
test.isnull().sum()

# train

In [ ]:
train = pd.read_csv("~/Documents/data/airbnb/train_users_2.csv")
train.head()

In [ ]:
train.isnull().sum()

# drop date_first_booking column

In [ ]:
train = train.drop(labels=['date_first_booking'], axis=1)
test = test.drop(labels=['date_first_booking'], axis=1)

# first_affiliate_tracked nan data manipulation

In [ ]:
train['first_affiliate_tracked'] = train.first_affiliate_tracked.replace(np.nan, "untracked")
test['first_affiliate_tracked'] = test.first_affiliate_tracked.replace(np.nan, "untracked")

# age

In [ ]:
print(len(train[train['age'] > 120]))

In [ ]:
train[train['age'] > 120]

In [ ]:
train_copy = train.copy()
train_copy = train_copy[train_copy['age'] <= 120]
train_copy.isnull().sum()

In [ ]:
train_copy = train.copy()
train_copy = train_copy.dropna()
train_copy = train_copy[train_copy['age'] <= 120]

ax1 = sns.distplot(train_copy['age'])
plt.show()
ax2 = sns.boxplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
plt.show()
# ax3 = sns.swarmplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
# plt.show()

# age nan value manipulation

- DBSCAN
- heatmap age vs related features

In [ ]:
# table = pivot_table(train, values='age', columns=['gender', 'signup_method', 'affiliate_provider', 'signup_app'])

In [ ]:
train_copy.describe()

In [ ]:
# #data type convert
# train_test["date_account_created"] =  pd.to_datetime(train_test["date_account_created"])
# train_test["date_first_booking"] =  pd.to_datetime(train_test["date_first_booking"])

# train_test["date_first_booking"] =  pd.to_datetime(train_test["date_first_booking"])
# train_test.info()

In [ ]:
# train_test = train_test.fillna(0)
# train_test.info()

In [ ]:
# vals = train_test.values
# X = vals[:piv_train]
# le = LabelEncoder()
# y = le.fit_transform(labels)   
# X_test = vals[piv_train:]

# random forest regression

In [52]:
import numpy as np

from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score

rng = np.random.RandomState(0)

dataset = load_boston()
X_full, y_full = dataset.data, dataset.target
n_samples = X_full.shape[0]
n_features = X_full.shape[1]

# Estimate the score on the entire dataset, with no missing values
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_full, y_full).mean()
print("Score with the entire dataset = %.2f" % score)

# Add missing values in 75% of the lines
missing_rate = 0.75
n_missing_samples = int(np.floor(n_samples * missing_rate))
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples,
                                      dtype=np.bool),
                             np.ones(n_missing_samples,
                                     dtype=np.bool)))
rng.shuffle(missing_samples)
missing_features = rng.randint(0, n_features, n_missing_samples)

# Estimate the score without the lines containing missing values
X_filtered = X_full[~missing_samples, :]
y_filtered = y_full[~missing_samples]
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_filtered, y_filtered).mean()
print("Score without the samples containing missing values = %.2f" % score)

# Estimate the score after imputation of the missing values
X_missing = X_full.copy()
X_missing[np.where(missing_samples)[0], missing_features] = 0
y_missing = y_full.copy()
estimator = Pipeline([("imputer", Imputer(missing_values=0,
                                          strategy="mean",
                                          axis=0)),
                      ("forest", RandomForestRegressor(random_state=0,
                                                       n_estimators=100))])
score = cross_val_score(estimator, X_missing, y_missing).mean()
print("Score after imputation of the missing values = %.2f" % score)

Score with the entire dataset = 0.56
Score without the samples containing missing values = 0.48
Score after imputation of the missing values = 0.57


In [ ]:
X_full1 = train.drop(labels=['age'], axis=1)
y_full1 = train['age']

In [ ]:
rng = np.random.RandomState(0)

dataset = load_boston()
X_full, y_full = dataset.data, dataset.target
n_samples = X_full.shape[0]
n_features = X_full.shape[1]

# Estimate the score on the entire dataset, with no missing values
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_full, y_full).mean()
print("Score with the entire dataset = %.2f" % score)

# Add missing values in 75% of the lines
missing_rate = 0.75
n_missing_samples = int(np.floor(n_samples * missing_rate))
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples,
                                      dtype=np.bool),
                             np.ones(n_missing_samples,
                                     dtype=np.bool)))
rng.shuffle(missing_samples)
missing_features = rng.randint(0, n_features, n_missing_samples)

# Estimate the score without the lines containing missing values
X_filtered = X_full[~missing_samples, :]
y_filtered = y_full[~missing_samples]
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_filtered, y_filtered).mean()
print("Score without the samples containing missing values = %.2f" % score)

# Estimate the score after imputation of the missing values
X_missing = X_full.copy()
X_missing[np.where(missing_samples)[0], missing_features] = 0
y_missing = y_full.copy()
estimator = Pipeline([("imputer", Imputer(missing_values=0,
                                          strategy="mean",
                                          axis=0)),
                      ("forest", RandomForestRegressor(random_state=0,
                                                       n_estimators=100))])
score = cross_val_score(estimator, X_missing, y_missing).mean()
print("Score after imputation of the missing values = %.2f" % score)

In [ ]:
# train_1 = train.drop(['country_destination'], axis=1)
# train_1.info()

In [ ]:
# y = train['country_destination']

In [ ]:
# X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
# regr = RandomForestRegressor(max_depth=2, random_state=0)
# regr.fit(train_1, y)

# print(regr.feature_importances_)
# print(regr.predict([[0, 0, 0, 0]]))


# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
train_test.info()

In [ ]:
data = np.random.rand(5, 10)  # 5 entities, each contains 10 features
label = np.random.randint(2, size=5)  # binary target
dtrain = xgb.DMatrix(data, label=label)

In [ ]:
dtrain = xgb.DMatrix(train_test, missing=-999.0)

# nueral network

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
clf.fit(train, train_no_target)

In [60]:
import pandas as pd

In [61]:
df_train = pd.read_csv("~/Documents/data/airbnb/train_users_2.csv")
df_test = pd.read_csv("~/Documents/data/airbnb/test_users.csv")
df_all = pd.concat([df_train, df_test])

In [62]:
# df_target = df_all["age"]

In [63]:
# df_all = df_all.drop("age", axis = 1)
df_all = df_all.drop("date_first_booking", axis = 1)

In [64]:
df_all["first_affiliate_tracked"].fillna("untracked", inplace = True)

In [65]:
df_all["date_account_created"] = pd.to_datetime(df_all["date_account_created"], format = "%Y-%m-%d")

In [66]:
df_all["timestamp_first_active"] = pd.to_datetime(df_all["timestamp_first_active"], format="%Y%m%d%H%M%S", errors='ignore')

In [67]:
df_all["ac_year"] = df_all["date_account_created"].apply(lambda x : x.year)
df_all["ac_month"] = df_all["date_account_created"].apply(lambda x : x.month)
df_all["ac_day"] = df_all["date_account_created"].apply(lambda x : x.day)

In [68]:
df_all["fa_year"] = df_all["timestamp_first_active"].apply(lambda x : x.year)
df_all["fa_month"] = df_all["timestamp_first_active"].apply(lambda x : x.month)
df_all["fa_day"] = df_all["timestamp_first_active"].apply(lambda x : x.day)
df_all["fa_hour"] = df_all["timestamp_first_active"].apply(lambda x : x.hour)
df_all["fa_minute"] = df_all["timestamp_first_active"].apply(lambda x : x.minute)
df_all["fa_second"] = df_all["timestamp_first_active"].apply(lambda x : x.second)

In [69]:
df_all = df_all.drop("date_account_created", axis = 1)
df_all = df_all.drop("timestamp_first_active", axis = 1)
df_all = df_all.drop("country_destination", axis = 1)

In [70]:
df_all = df_all.sort_values(by='age', na_position='first').reset_index(drop=True)
df_all.isnull().sum()

affiliate_channel               0
affiliate_provider              0
age                        116866
first_affiliate_tracked         0
first_browser                   0
first_device_type               0
gender                          0
id                              0
language                        0
signup_app                      0
signup_flow                     0
signup_method                   0
ac_year                         0
ac_month                        0
ac_day                          0
fa_year                         0
fa_month                        0
fa_day                          0
fa_hour                         0
fa_minute                       0
fa_second                       0
dtype: int64

In [71]:
df_all.head()

,affiliate_channel,affiliate_provider,age,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,...,signup_method,ac_year,ac_month,ac_day,fa_year,fa_month,fa_day,fa_hour,fa_minute,fa_second
0,direct,direct,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,...,facebook,2010,6,28,2009,3,19,4,32,55
1,other,other,NaN,omg,Chrome,Mac Desktop,-unknown-,osr2jwljor,en,Web,...,basic,2010,1,1,2010,1,1,21,56,19
2,direct,direct,NaN,untracked,-unknown-,Other/Unknown,-unknown-,k6np330cm1,en,Web,...,basic,2010,1,5,2010,1,5,6,8,59
3,other,craigslist,NaN,untracked,-unknown-,Other/Unknown,-unknown-,2dwbwkx056,en,Web,...,basic,2010,1,7,2010,1,7,21,51,25
4,seo,facebook,NaN,untracked,-unknown-,Other/Unknown,-unknown-,cxlg85pg1r,en,Web,...,basic,2010,1,8,2010,1,8,1,56,41


In [79]:
df_target = df_all.iloc[116866: , 2]
len(df_target)

158681

In [80]:
df_target

116866       1.0
116867       1.0
116868       1.0
116869       1.0
116870       2.0
116871       2.0
116872       2.0
116873       2.0
116874       2.0
116875       2.0
116876       2.0
116877       4.0
116878       4.0
116879       4.0
116880       5.0
116881       5.0
116882       5.0
116883       5.0
116884       5.0
116885       5.0
116886       5.0
116887       5.0
116888       5.0
116889       5.0
116890       5.0
116891       5.0
116892       5.0
116893       5.0
116894       5.0
116895       5.0
           ...  
275517    2014.0
275518    2014.0
275519    2014.0
275520    2014.0
275521    2014.0
275522    2014.0
275523    2014.0
275524    2014.0
275525    2014.0
275526    2014.0
275527    2014.0
275528    2014.0
275529    2014.0
275530    2014.0
275531    2014.0
275532    2014.0
275533    2014.0
275534    2014.0
275535    2014.0
275536    2014.0
275537    2014.0
275538    2014.0
275539    2014.0
275540    2014.0
275541    2014.0
275542    2014.0
275543    2014.0
275544    2014

In [81]:
df_all.iloc[116860:116880, 2]

116860    NaN
116861    NaN
116862    NaN
116863    NaN
116864    NaN
116865    NaN
116866    1.0
116867    1.0
116868    1.0
116869    1.0
116870    2.0
116871    2.0
116872    2.0
116873    2.0
116874    2.0
116875    2.0
116876    2.0
116877    4.0
116878    4.0
116879    4.0
Name: age, dtype: float64

In [82]:
df_target.tail()

275542    2014.0
275543    2014.0
275544    2014.0
275545    2014.0
275546    2014.0
Name: age, dtype: float64

In [83]:
df_all = df_all.drop("age", axis = 1)

In [84]:
df_all = df_all.drop("id", axis = 1)

In [85]:
df_all = pd.get_dummies(df_all)

In [97]:
df_train = df_all.iloc[116866:]
df_test = df_all.iloc[:116866]

In [98]:
df_train.shape, df_test.shape

((158681, 145), (116866, 145))

In [99]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [100]:
y = df_target
X = df_train
X_test = df_test

In [101]:
y.head()

116866    1.0
116867    1.0
116868    1.0
116869    1.0
116870    2.0
Name: age, dtype: float64

In [102]:
>>> from sklearn.datasets import load_boston
>>> from sklearn.model_selection import cross_val_score
>>> from sklearn.tree import DecisionTreeRegressor

In [117]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [124]:
models = RandomForestClassifier(max_depth=2)
models.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [127]:
y_pre = models.predict(X_test)

In [128]:
aaa = pd.DataFrame(y_pre)

In [103]:
regressor = DecisionTreeRegressor(random_state=0, max_depth=2)

In [104]:
cross_val_score(regressor, X, y, cv=10)

array([-1.47730022e+02, -1.02706537e+03, -1.09989211e+03, -9.69063293e+02,
       -8.28206682e+02, -5.26949033e+02, -2.22357763e+02, -6.96507109e+01,
       -1.98994986e+01, -9.88030148e-02])

In [109]:
regressor.fit(X,y)

In [113]:
y_pred = regressor.predict(X_test)

In [129]:
aaa.describe()

,0
count,116866.000000
mean,29.330943
std,0.929990
min,25.000000
25%,28.000000
50%,30.000000
75%,30.000000
max,45.000000


In [116]:
pd.Series(y_pred).describe()

count    116866.000000
mean         53.302472
std          10.105449
min          34.691005
25%          44.419362
50%          44.419362
75%          64.479344
max          64.479344
dtype: float64